In [1]:
stock = 'INTC'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    51.596218
2018-07-23    51.993801
2018-07-24    51.864586
2018-07-25    52.113075
2018-07-26    51.844707
2018-07-27    47.391788
2018-07-30    47.401726
2018-07-31    47.809246
2018-08-01    48.514957
2018-08-02    49.180904
2018-08-03    49.330002
2018-08-06    49.299999
2018-08-07    49.700001
2018-08-08    49.959999
2018-08-09    50.139999
2018-08-10    48.849998
2018-08-13    48.450001
2018-08-14    48.119999
2018-08-15    47.459999
2018-08-16    47.169998
2018-08-17    47.099998
2018-08-20    46.500000
2018-08-21    47.619999
2018-08-22    47.049999
2018-08-23    46.980000
2018-08-24    47.660000
2018-08-27    48.349998
2018-08-28    48.570000
2018-08-29    48.750000
2018-08-30    48.240002
                ...    
2018-09-17    45.419998
2018-09-18    46.099998
2018-09-19    46.150002
2018-09-20    47.200001
2018-09-21    46.660000
2018-09-24    46.910000
2018-09-25    45.910000
2018-09-26    45.700001
2018-09-27    45.880001
2018-09-28    47.290001
2018-10-01 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


47.22277292576102
[46.81709813 47.30954534 47.99042693 48.41705919 45.57973504]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")